In [1]:
#-------------------[IMPORT MODULES]-----------------------#
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
import joblib
import os
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

#-------------------[STOCK DATABASE]-----------------------#
STOCK_DATABASE = {
    'AAPL': 'Apple Inc.',
    'MSFT': 'Microsoft Corporation',
    'GOOGL': 'Alphabet Inc.',
    'AMZN': 'Amazon.com Inc.',
    'TSLA': 'Tesla Inc.',
    'META': 'Meta Platforms Inc.',
    'NVDA': 'NVIDIA Corporation',
    'JPM': 'JPMorgan Chase & Co.',
    'V': 'Visa Inc.',
    'WMT': 'Walmart Inc.',
    'DIS': 'The Walt Disney Company',
    'NFLX': 'Netflix Inc.',
    'INTC': 'Intel Corporation',
    'AMD': 'Advanced Micro Devices',
    'PYPL': 'PayPal Holdings Inc.'
}

def load_stock_list():
    """Load stock list from database"""
    return STOCK_DATABASE

#-------------------[TECHNICAL INDICATORS FUNCTIONS]-----------------------#
def calculate_technical_indicators(data):
    """Calculate comprehensive technical indicators"""
    df = data.copy()
    
    # Flatten MultiIndex columns if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    # Ensure we're working with Series
    close = df['Close'].squeeze()
    high = df['High'].squeeze()
    low = df['Low'].squeeze()
    volume = df['Volume'].squeeze()
    
    # Moving Averages
    for period in [7, 20, 50, 100, 200]:
        df[f'MA_{period}'] = close.rolling(period).mean()
        df[f'MA_{period}_slope'] = df[f'MA_{period}'].diff(5)
    
    # Exponential Moving Averages
    for period in [12, 26, 50]:
        df[f'EMA_{period}'] = close.ewm(span=period, adjust=False).mean()
    
    # MACD
    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # RSI (Multiple periods)
    for period in [14, 21]:
        delta = close.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / (loss + 1e-10)  # Avoid division by zero
        df[f'RSI_{period}'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    for period in [20, 50]:
        bb_middle = close.rolling(period).mean()
        bb_std = close.rolling(period).std()
        df[f'BB_Middle_{period}'] = bb_middle
        df[f'BB_Upper_{period}'] = bb_middle + (bb_std * 2)
        df[f'BB_Lower_{period}'] = bb_middle - (bb_std * 2)
        df[f'BB_Width_{period}'] = (df[f'BB_Upper_{period}'] - df[f'BB_Lower_{period}']) / bb_middle
        df[f'BB_Position_{period}'] = (close - df[f'BB_Lower_{period}']) / (df[f'BB_Upper_{period}'] - df[f'BB_Lower_{period}'] + 1e-10)
    
    # Stochastic Oscillator
    low_14 = low.rolling(14).min()
    high_14 = high.rolling(14).max()
    df['Stochastic_%K'] = 100 * (close - low_14) / (high_14 - low_14 + 1e-10)
    df['Stochastic_%D'] = df['Stochastic_%K'].rolling(3).mean()
    
    # ATR (Average True Range)
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    df['ATR_14'] = true_range.rolling(14).mean()
    
    # Volume indicators
    df['Volume_MA_20'] = volume.rolling(20).mean()
    df['Volume_MA_50'] = volume.rolling(50).mean()
    df['Volume_Ratio'] = volume / (df['Volume_MA_20'] + 1)
    
    # OBV (On-Balance Volume)
    df['OBV'] = (np.sign(close.diff()) * volume).fillna(0).cumsum()
    df['OBV_MA'] = df['OBV'].rolling(20).mean()
    
    # Money Flow Index
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    
    positive_flow = money_flow.where(typical_price > typical_price.shift(1), 0).rolling(14).sum()
    negative_flow = money_flow.where(typical_price < typical_price.shift(1), 0).rolling(14).sum()
    df['MFI'] = 100 - (100 / (1 + positive_flow / (negative_flow + 1e-10)))
    
    # ADX (Average Directional Index)
    plus_dm = high.diff()
    minus_dm = low.diff().abs()
    plus_dm = plus_dm.where((plus_dm > minus_dm) & (plus_dm > 0), 0)
    minus_dm = minus_dm.where((minus_dm > plus_dm.abs()) & (minus_dm > 0), 0)
    
    tr = true_range.rolling(14).sum()
    plus_di = 100 * (plus_dm.rolling(14).sum() / tr)
    minus_di = 100 * (minus_dm.rolling(14).sum() / tr)
    
    dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di + 1e-10)
    df['ADX'] = dx.rolling(14).mean()
    
    return df

#-------------------[ENHANCED STOCK PREDICTION CLASS]-----------------------#
class EnhancedStockPricePredictor:
    def __init__(self):
        self.models = {}
        self.ensemble_weights = {}
        self.scaler = RobustScaler()  # More robust to outliers
        self.feature_columns = None
        self.feature_importance = None
        self.is_trained = False
        self.ticker = None
        self.company_name = None
        self.training_metrics = {}
        self.validation_results = {}
    
    def prepare_features(self, data):
        """Enhanced feature engineering with advanced technical analysis"""
        df = data.copy()
        
        # Flatten MultiIndex columns if present
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
            
        # Ensure we're working with Series
        close = df['Close'].squeeze()
        open_price = df['Open'].squeeze()
        high = df['High'].squeeze()
        low = df['Low'].squeeze()
        volume = df['Volume'].squeeze()
        
        # Price-based features
        df['Price_Range'] = high - low
        df['Price_Change'] = close - open_price
        df['Price_Change_Pct'] = (close - open_price) / (open_price + 1e-10) * 100
        df['Close_Open_Ratio'] = close / (open_price + 1e-10)
        df['High_Low_Ratio'] = high / (low + 1e-10)
        df['High_Close_Ratio'] = high / (close + 1e-10)
        df['Low_Close_Ratio'] = low / (close + 1e-10)
        
        # Intraday features
        df['Upper_Shadow'] = high - np.maximum(close, open_price)
        df['Lower_Shadow'] = np.minimum(close, open_price) - low
        df['Body_Size'] = np.abs(close - open_price)
        
        # Moving Averages and crossovers
        for window in [5, 7, 10, 15, 20, 30, 50, 100, 200]:
            ma = close.rolling(window).mean()
            df[f'MA_{window}'] = ma
            df[f'Price_vs_MA_{window}'] = (close - ma) / (ma + 1e-10) * 100
            df[f'MA_{window}_Trend'] = ma.diff(5)
        
        # MA crossovers
        df['MA_5_20_Cross'] = (df['MA_5'] - df['MA_20']) / (df['MA_20'] + 1e-10)
        df['MA_20_50_Cross'] = (df['MA_20'] - df['MA_50']) / (df['MA_50'] + 1e-10)
        df['MA_50_200_Cross'] = (df['MA_50'] - df['MA_200']) / (df['MA_200'] + 1e-10)
        
        # Exponential Moving Averages
        for span in [8, 12, 21, 26, 50, 100]:
            ema = close.ewm(span=span, adjust=False).mean()
            df[f'EMA_{span}'] = ema
            df[f'Price_vs_EMA_{span}'] = (close - ema) / (ema + 1e-10) * 100
        
        # MACD variations
        ema_12 = close.ewm(span=12, adjust=False).mean()
        ema_26 = close.ewm(span=26, adjust=False).mean()
        macd = ema_12 - ema_26
        df['MACD'] = macd
        df['MACD_Signal'] = macd.ewm(span=9, adjust=False).mean()
        df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
        df['MACD_Histogram_Change'] = df['MACD_Histogram'].diff()
        
        # RSI for multiple periods
        for period in [9, 14, 21, 28]:
            delta = close.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / (loss + 1e-10)
            df[f'RSI_{period}'] = 100 - (100 / (1 + rs))
        
        # Bollinger Bands
        for period in [20, 50]:
            bb_middle = close.rolling(period).mean()
            bb_std = close.rolling(period).std()
            df[f'BB_Middle_{period}'] = bb_middle
            df[f'BB_Upper_{period}'] = bb_middle + (bb_std * 2)
            df[f'BB_Lower_{period}'] = bb_middle - (bb_std * 2)
            df[f'BB_Width_{period}'] = (df[f'BB_Upper_{period}'] - df[f'BB_Lower_{period}']) / (bb_middle + 1e-10)
            df[f'BB_Position_{period}'] = (close - df[f'BB_Lower_{period}']) / (df[f'BB_Upper_{period}'] - df[f'BB_Lower_{period}'] + 1e-10)
        
        # Stochastic Oscillator
        low_14 = low.rolling(14).min()
        high_14 = high.rolling(14).max()
        df['Stochastic_%K'] = 100 * (close - low_14) / (high_14 - low_14 + 1e-10)
        df['Stochastic_%D'] = df['Stochastic_%K'].rolling(3).mean()
        
        # Volume indicators
        for window in [10, 20, 50]:
            volume_ma = volume.rolling(window).mean()
            df[f'Volume_MA_{window}'] = volume_ma
            df[f'Volume_Ratio_{window}'] = volume / (volume_ma + 1)
        
        # OBV
        df['OBV'] = (np.sign(close.diff()) * volume).fillna(0).cumsum()
        df['OBV_MA'] = df['OBV'].rolling(20).mean()
        df['OBV_Trend'] = df['OBV'] - df['OBV_MA']
        
        # Momentum indicators
        for period in [5, 10, 15, 20, 30]:
            df[f'Momentum_{period}'] = close.diff(period)
            df[f'ROC_{period}'] = ((close - close.shift(period)) / (close.shift(period) + 1e-10)) * 100
        
        # Volatility measures
        for period in [10, 20, 30]:
            df[f'Volatility_{period}'] = close.rolling(period).std()
            df[f'Volatility_{period}_Norm'] = df[f'Volatility_{period}'] / (close + 1e-10) * 100
        
        # Historical Volatility (Parkinson)
        df['Parkinson_Volatility'] = np.sqrt(1/(4*np.log(2)) * ((np.log(high/low))**2).rolling(20).mean())
        
        # Daily returns and statistics
        df['Daily_Return'] = close.pct_change()
        df['Daily_Return_Squared'] = df['Daily_Return'] ** 2
        for period in [5, 10, 20]:
            df[f'Return_Mean_{period}'] = df['Daily_Return'].rolling(period).mean()
            df[f'Return_Std_{period}'] = df['Daily_Return'].rolling(period).std()
        
        # Price patterns
        df['Higher_High'] = (high > high.shift(1)).astype(int)
        df['Lower_Low'] = (low < low.shift(1)).astype(int)
        df['Higher_Close'] = (close > close.shift(1)).astype(int)
        
        # Lag features (multiple periods)
        for lag in [1, 2, 3, 5, 7, 10]:
            df[f'Close_Lag_{lag}'] = close.shift(lag)
            df[f'Volume_Lag_{lag}'] = volume.shift(lag)
            df[f'Return_Lag_{lag}'] = df['Daily_Return'].shift(lag)
        
        # Rolling statistics
        for window in [5, 10, 20]:
            df[f'Close_Max_{window}'] = close.rolling(window).max()
            df[f'Close_Min_{window}'] = close.rolling(window).min()
            df[f'Close_Std_{window}'] = close.rolling(window).std()
            df[f'Price_Position_{window}'] = (close - df[f'Close_Min_{window}']) / (df[f'Close_Max_{window}'] - df[f'Close_Min_{window}'] + 1e-10)
        
        # Target variables for different time horizons
        df['Target_1_Day'] = close.shift(-1)
        df['Target_5_Days'] = close.shift(-5)
        df['Target_21_Days'] = close.shift(-21)
        
        # Target percentage changes
        df['Target_1_Day_Pct'] = ((df['Target_1_Day'] - close) / close) * 100
        df['Target_5_Days_Pct'] = ((df['Target_5_Days'] - close) / close) * 100
        df['Target_21_Days_Pct'] = ((df['Target_21_Days'] - close) / close) * 100
        
        return df
    
    def train_ensemble_model(self, data, target_horizon='1_Day'):
        """Train an ensemble of models for better predictions"""
        print(f"🔄 Training ensemble model for {target_horizon.replace('_', ' ')} prediction...")
        
        # Prepare features
        data_with_features = self.prepare_features(data)
        data_clean = data_with_features.dropna()
        
        if len(data_clean) < 100:
            print(f"❌ Insufficient data for training (need at least 100 samples, got {len(data_clean)})")
            return False
        
        # Define feature columns (select most relevant features)
        feature_candidates = [col for col in data_clean.columns 
                            if col not in ['Target_1_Day', 'Target_5_Days', 'Target_21_Days',
                                         'Target_1_Day_Pct', 'Target_5_Days_Pct', 'Target_21_Days_Pct']]
        
        # Remove columns with too many NaN or inf values
        available_features = []
        for col in feature_candidates:
            if data_clean[col].replace([np.inf, -np.inf], np.nan).notna().sum() > len(data_clean) * 0.8:
                available_features.append(col)
        
        target_column = f'Target_{target_horizon}'
        
        if target_column not in data_clean.columns:
            print(f"❌ Target column {target_column} not found!")
            return False
        
        # Prepare data
        X = data_clean[available_features].replace([np.inf, -np.inf], np.nan).fillna(method='ffill').fillna(method='bfill')
        y = data_clean[target_column]
        
        # Time series split for validation
        tscv = TimeSeriesSplit(n_splits=5)
        
        # Store all validation scores
        all_scores = {'rf': [], 'gb': [], 'ridge': []}
        
        for train_idx, val_idx in tscv.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Scale features
            X_train_scaled = self.scaler.fit_transform(X_train)
            X_val_scaled = self.scaler.transform(X_val)
            
            # Train multiple models
            models = {
                'rf': RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5,
                                           min_samples_leaf=2, random_state=42, n_jobs=-1),
                'gb': GradientBoostingRegressor(n_estimators=200, max_depth=5, learning_rate=0.05,
                                               subsample=0.8, random_state=42),
                'ridge': Ridge(alpha=1.0, random_state=42)
            }
            
            for name, model in models.items():
                model.fit(X_train_scaled, y_train)
                val_pred = model.predict(X_val_scaled)
                mae = mean_absolute_error(y_val, val_pred)
                all_scores[name].append(mae)
        
        # Calculate average scores and weights
        avg_scores = {name: np.mean(scores) for name, scores in all_scores.items()}
        inverse_scores = {name: 1/score for name, score in avg_scores.items()}
        total_inverse = sum(inverse_scores.values())
        weights = {name: inv/total_inverse for name, inv in inverse_scores.items()}
        
        # Train final models on all data
        X_scaled = self.scaler.fit_transform(X)
        
        final_models = {
            'rf': RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5,
                                       min_samples_leaf=2, random_state=42, n_jobs=-1),
            'gb': GradientBoostingRegressor(n_estimators=200, max_depth=5, learning_rate=0.05,
                                           subsample=0.8, random_state=42),
            'ridge': Ridge(alpha=1.0, random_state=42)
        }
        
        for name, model in final_models.items():
            model.fit(X_scaled, y)
        
        # Store models and weights
        self.models[target_horizon] = final_models
        self.ensemble_weights[target_horizon] = weights
        
        # Evaluate ensemble
        ensemble_pred = np.zeros(len(y))
        for name, model in final_models.items():
            ensemble_pred += model.predict(X_scaled) * weights[name]
        
        mae = mean_absolute_error(y, ensemble_pred)
        rmse = np.sqrt(mean_squared_error(y, ensemble_pred))
        r2 = r2_score(y, ensemble_pred)
        mape = mean_absolute_percentage_error(y, ensemble_pred)
        
        # Calculate directional accuracy
        actual_direction = (y.diff() > 0).astype(int)
        pred_direction = (pd.Series(ensemble_pred).diff() > 0).astype(int)
        directional_accuracy = (actual_direction == pred_direction).sum() / len(actual_direction) * 100
        
        # Store metrics
        self.training_metrics[target_horizon] = {
            'mae': mae,
            'rmse': rmse,
            'r2_score': r2,
            'mape': mape,
            'directional_accuracy': directional_accuracy,
            'ensemble_weights': weights,
            'model_scores': avg_scores
        }
        
        # Feature importance (from Random Forest)
        if hasattr(final_models['rf'], 'feature_importances_'):
            feature_imp = pd.DataFrame({
                'feature': available_features,
                'importance': final_models['rf'].feature_importances_
            }).sort_values('importance', ascending=False)
            self.feature_importance = feature_imp
        
        print(f"📈 Ensemble Model Performance ({target_horizon.replace('_', ' ')}):")
        print(f"   MAE: ${mae:.2f} | RMSE: ${rmse:.2f} | R²: {r2:.4f} | MAPE: {mape:.2f}%")
        print(f"   Directional Accuracy: {directional_accuracy:.1f}%")
        print(f"   Ensemble Weights: RF={weights['rf']:.2f}, GB={weights['gb']:.2f}, Ridge={weights['ridge']:.2f}")
        
        self.is_trained = True
        self.feature_columns = available_features
        
        return True
    
    def predict_future(self, data, target_horizon='1_Day'):
        """Predict future stock prices using ensemble"""
        if not self.is_trained or target_horizon not in self.models:
            print(f"❌ Model for {target_horizon} not trained!")
            return None
        
        # Prepare features
        data_with_features = self.prepare_features(data)
        latest_data = data_with_features[self.feature_columns].iloc[-1:].replace([np.inf, -np.inf], np.nan).fillna(method='ffill').fillna(0)
        
        # Scale features
        latest_data_scaled = self.scaler.transform(latest_data)
        
        # Ensemble prediction
        ensemble_pred = 0
        for name, model in self.models[target_horizon].items():
            pred = model.predict(latest_data_scaled)[0]
            weight = self.ensemble_weights[target_horizon][name]
            ensemble_pred += pred * weight
        
        # Get current price
        if isinstance(data['Close'], pd.DataFrame):
            current_price = data['Close'].iloc[-1].values[0]
        else:
            current_price = data['Close'].iloc[-1]
        
        # Calculate confidence interval (using prediction std)
        predictions = [model.predict(latest_data_scaled)[0] for model in self.models[target_horizon].values()]
        pred_std = np.std(predictions)
        
        return {
            'current_price': current_price,
            'predicted_price': ensemble_pred,
            'price_change': ensemble_pred - current_price,
            'price_change_pct': ((ensemble_pred / current_price) - 1) * 100,
            'confidence_lower': ensemble_pred - 1.96 * pred_std,
            'confidence_upper': ensemble_pred + 1.96 * pred_std,
            'prediction_std': pred_std
        }

#-------------------[FIND STOCK]-----------------------#
def find_stock(search_term):
    """Find stock in database"""
    matches = {}
    stock_list = load_stock_list()
    
    for symbol, name in stock_list.items():
        if (search_term.upper() in symbol.upper() or 
            search_term.lower() in name.lower()):
            matches[symbol] = name
    
    return matches

#-------------------[CREATE DIRECTORIES]-----------------------#
def create_directories():
    """Create necessary directories"""
    directories = ['Training/stock_data', 'Training/models', 'Training/charts']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
    print("✓ All directories created successfully")

#-------------------[ENHANCED VISUALIZATION]-----------------------#
def visualize_with_predictions(data, ticker, company_name, predictions, predictor):
    """Create comprehensive visualizations with prediction results"""
    print("\n📊 Generating enhanced visualizations with predictions...")
    
    # Calculate technical indicators
    data_with_indicators = calculate_technical_indicators(data)
    data_clean = data_with_indicators.dropna()
    
    # Get current price
    if isinstance(data['Close'], pd.DataFrame):
        current_price = data['Close'].iloc[-1].values[0]
    else:
        current_price = data['Close'].iloc[-1]
    
    # Create figure with subplots
    fig = plt.figure(figsize=(22, 18))
    gs = fig.add_gridspec(6, 2, hspace=0.35, wspace=0.3)
    
    # 1. Main Price Chart with Predictions and Confidence Intervals
    ax1 = fig.add_subplot(gs[0:2, :])
    ax1.plot(data_clean.index, data_clean['Close'], label='Historical Price', color='blue', linewidth=2.5)
    ax1.plot(data_clean.index, data_clean['MA_50'], label='MA 50', color='red', linewidth=1.5, alpha=0.7)
    ax1.plot(data_clean.index, data_clean['MA_200'], label='MA 200', color='green', linewidth=1.5, alpha=0.7)
    
    # Add Bollinger Bands
    ax1.fill_between(data_clean.index, data_clean['BB_Upper_20'], data_clean['BB_Lower_20'], 
                     alpha=0.1, color='gray', label='Bollinger Bands')
    
    # Add prediction markers with confidence intervals
    last_date = data_clean.index[-1]
    ax1.scatter(last_date, current_price, color='blue', s=250, zorder=5, marker='o', 
               edgecolors='black', linewidths=2, label='Current Price')
    
    colors = {'1 Day': 'orange', '1 Week': 'purple', '1 Month': 'red'}
    offsets = {'1 Day': 1, '1 Week': 5, '1 Month': 21}
    
    for horizon, pred in predictions.items():
        future_date = last_date + timedelta(days=offsets[horizon])
        
        # Main prediction
        ax1.scatter(future_date, pred['predicted_price'], color=colors[horizon], 
                   s=250, zorder=5, marker='*', edgecolors='black', linewidths=2,
                   label=f'{horizon} Prediction')
        
        # Confidence interval
        ax1.errorbar(future_date, pred['predicted_price'], 
                    yerr=[[pred['predicted_price'] - pred['confidence_lower']], 
                          [pred['confidence_upper'] - pred['predicted_price']]], 
                    fmt='none', ecolor=colors[horizon], elinewidth=2, capsize=5, alpha=0.6)
        
        # Prediction line
        ax1.plot([last_date, future_date], [current_price, pred['predicted_price']], 
                color=colors[horizon], linestyle='--', linewidth=2.5, alpha=0.7)
        
        # Enhanced annotation
        annotation_text = f"${pred['predicted_price']:.2f}\n"
        annotation_text += f"{pred['price_change_pct']:+.2f}%\n"
        annotation_text += f"95% CI: ${pred['confidence_lower']:.2f}-${pred['confidence_upper']:.2f}"
        
        ax1.annotate(annotation_text,
                    xy=(future_date, pred['predicted_price']),
                    xytext=(15, 15), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.7', fc=colors[horizon], alpha=0.8, edgecolor='black', linewidth=1.5),
                    fontsize=9, fontweight='bold', color='white',
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', lw=1.5))
    
    ax1.set_title(f'{ticker} - {company_name} - Price Analysis with AI Predictions & Confidence Intervals', 
                 fontsize=18, fontweight='bold', pad=25)
    ax1.legend(loc='upper left', fontsize=11, framealpha=0.9)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.set_ylabel('Price ($)', fontsize=14, fontweight='bold')
    
    # 2. Volume Analysis
    ax2 = fig.add_subplot(gs[2, :])
    colors_vol = ['red' if data_clean['Close'].iloc[i] < data_clean['Close'].iloc[i-1] else 'green' 
                  for i in range(1, len(data_clean))]
    colors_vol = ['gray'] + colors_vol
    
    ax2.bar(data_clean.index, data_clean['Volume'], color=colors_vol, alpha=0.6, label='Volume')
    ax2.plot(data_clean.index, data_clean['Volume_MA_20'], color='blue', linewidth=2.5, label='Volume MA 20')
    ax2.plot(data_clean.index, data_clean['Volume_MA_50'], color='red', linewidth=2, label='Volume MA 50', alpha=0.7)
    ax2.set_title('Trading Volume Analysis with Moving Averages', fontsize=14, fontweight='bold')
    ax2.legend(loc='upper left', fontsize=10)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.set_ylabel('Volume', fontsize=12, fontweight='bold')
    
    # 3. RSI Indicator
    ax3 = fig.add_subplot(gs[3, 0])
    ax3.plot(data_clean.index, data_clean['RSI_14'], color='purple', linewidth=2.5, label='RSI (14)')
    ax3.axhline(70, color='red', linestyle='--', alpha=0.7, linewidth=2, label='Overbought (70)')
    ax3.axhline(30, color='green', linestyle='--', alpha=0.7, linewidth=2, label='Oversold (30)')
    ax3.axhline(50, color='gray', linestyle=':', alpha=0.5, linewidth=1.5)
    ax3.fill_between(data_clean.index, 30, 70, alpha=0.1, color='gray')
    ax3.set_title('RSI (Relative Strength Index)', fontsize=13, fontweight='bold')
    ax3.set_ylim(0, 100)
    ax3.legend(loc='upper left', fontsize=9)
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.set_ylabel('RSI', fontsize=11, fontweight='bold')
    
    # 4. MACD Indicator
    ax4 = fig.add_subplot(gs[3, 1])
    ax4.plot(data_clean.index, data_clean['MACD'], color='blue', linewidth=2.5, label='MACD')
    ax4.plot(data_clean.index, data_clean['MACD_Signal'], color='red', linewidth=2.5, label='Signal Line')
    
    # Color histogram bars based on value
    colors_macd = ['green' if val > 0 else 'red' for val in data_clean['MACD_Histogram']]
    ax4.bar(data_clean.index, data_clean['MACD_Histogram'], color=colors_macd, alpha=0.4, label='Histogram')
    ax4.axhline(0, color='black', linestyle='-', linewidth=1)
    ax4.set_title('MACD (Moving Average Convergence Divergence)', fontsize=13, fontweight='bold')
    ax4.legend(loc='upper left', fontsize=9)
    ax4.grid(True, alpha=0.3, linestyle='--')
    ax4.set_ylabel('MACD', fontsize=11, fontweight='bold')
    
    # 5. Stochastic Oscillator
    ax5 = fig.add_subplot(gs[4, 0])
    ax5.plot(data_clean.index, data_clean['Stochastic_%K'], color='blue', linewidth=2, label='%K')
    ax5.plot(data_clean.index, data_clean['Stochastic_%D'], color='red', linewidth=2, label='%D')
    ax5.axhline(80, color='red', linestyle='--', alpha=0.6, linewidth=1.5)
    ax5.axhline(20, color='green', linestyle='--', alpha=0.6, linewidth=1.5)
    ax5.fill_between(data_clean.index, 20, 80, alpha=0.1, color='gray')
    ax5.set_title('Stochastic Oscillator', fontsize=13, fontweight='bold')
    ax5.set_ylim(0, 100)
    ax5.legend(loc='upper left', fontsize=9)
    ax5.grid(True, alpha=0.3, linestyle='--')
    ax5.set_ylabel('Stochastic', fontsize=11, fontweight='bold')
    
    # 6. ADX (Trend Strength)
    ax6 = fig.add_subplot(gs[4, 1])
    ax6.plot(data_clean.index, data_clean['ADX'], color='purple', linewidth=2.5, label='ADX')
    ax6.axhline(25, color='orange', linestyle='--', alpha=0.7, linewidth=2, label='Strong Trend (25)')
    ax6.axhline(50, color='red', linestyle='--', alpha=0.7, linewidth=2, label='Very Strong (50)')
    ax6.fill_between(data_clean.index, 25, 100, alpha=0.1, color='green')
    ax6.set_title('ADX (Average Directional Index) - Trend Strength', fontsize=13, fontweight='bold')
    ax6.set_ylim(0, 100)
    ax6.legend(loc='upper left', fontsize=9)
    ax6.grid(True, alpha=0.3, linestyle='--')
    ax6.set_ylabel('ADX', fontsize=11, fontweight='bold')
    
    # 7. Comprehensive Prediction Summary
    ax7 = fig.add_subplot(gs[5, :])
    ax7.axis('off')
    
    # Create detailed summary
    summary_text = f"🤖 ENHANCED AI PREDICTION SUMMARY - {ticker} ({company_name})\n\n"
    summary_text += f"{'='*90}\n"
    summary_text += f"💰 CURRENT MARKET DATA\n"
    summary_text += f"{'='*90}\n"
    summary_text += f"Current Price: ${current_price:.2f} | "
    summary_text += f"Analysis Date: {last_date.strftime('%Y-%m-%d')} | "
    summary_text += f"Data Points: {len(data_clean)}\n\n"
    
    summary_text += f"{'='*90}\n"
    summary_text += f"📈 AI PREDICTIONS WITH CONFIDENCE INTERVALS (95% CI)\n"
    summary_text += f"{'='*90}\n"
    
    for horizon, pred in predictions.items():
        change_emoji = "🚀" if pred['price_change_pct'] > 0 else "📉"
        summary_text += f"\n{change_emoji} {horizon.upper()}:\n"
        summary_text += f"   Target Price: ${pred['predicted_price']:.2f} "
        summary_text += f"(Change: ${pred['price_change']:+.2f} | {pred['price_change_pct']:+.2f}%)\n"
        summary_text += f"   95% Confidence: ${pred['confidence_lower']:.2f} - ${pred['confidence_upper']:.2f} "
        summary_text += f"(±${pred['prediction_std']:.2f})\n"
        
        # Enhanced recommendation logic
        pct = pred['price_change_pct']
        if pct > 10:
            recommendation = "STRONG BUY 🚀🚀🚀"
            action = "High conviction buy opportunity"
        elif pct > 5:
            recommendation = "STRONG BUY 🚀🚀"
            action = "Excellent buy signal"
        elif pct > 2:
            recommendation = "BUY 📈"
            action = "Good entry point"
        elif pct > -2:
            recommendation = "HOLD ⚖️"
            action = "Maintain current position"
        elif pct > -5:
            recommendation = "SELL 📉"
            action = "Consider reducing position"
        elif pct > -10:
            recommendation = "STRONG SELL 🔻🔻"
            action = "Exit position recommended"
        else:
            recommendation = "STRONG SELL 🔻🔻🔻"
            action = "Immediate exit recommended"
        
        summary_text += f"   → {recommendation} - {action}\n"
    
    summary_text += f"\n{'='*90}\n"
    summary_text += f"📊 MODEL PERFORMANCE METRICS\n"
    summary_text += f"{'='*90}\n"
    
    for horizon_key, metrics in predictor.training_metrics.items():
        horizon_name = horizon_key.replace('_', ' ')
        summary_text += f"\n{horizon_name}:\n"
        summary_text += f"   MAE: ${metrics['mae']:.2f} | "
        summary_text += f"RMSE: ${metrics['rmse']:.2f} | "
        summary_text += f"R²: {metrics['r2_score']:.3f} | "
        summary_text += f"MAPE: {metrics['mape']:.2f}%\n"
        summary_text += f"   Directional Accuracy: {metrics['directional_accuracy']:.1f}% | "
        summary_text += f"Ensemble: RF={metrics['ensemble_weights']['rf']:.2f}, "
        summary_text += f"GB={metrics['ensemble_weights']['gb']:.2f}, "
        summary_text += f"Ridge={metrics['ensemble_weights']['ridge']:.2f}\n"
    
    summary_text += f"\n{'='*90}\n"
    summary_text += "⚠️  DISCLAIMER: Predictions are based on historical patterns and technical analysis.\n"
    summary_text += "    Past performance does not guarantee future results. Always conduct your own research.\n"
    summary_text += f"{'='*90}\n"
    
    ax7.text(0.5, 0.5, summary_text, transform=ax7.transAxes,
            fontsize=9.5, verticalalignment='center', horizontalalignment='center',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9, 
                     edgecolor='darkblue', linewidth=2, pad=1.5),
            fontfamily='monospace', fontweight='bold')
    
    plt.savefig(f'Training/charts/{ticker}_enhanced_analysis.png', dpi=300, bbox_inches='tight')
    print(f"✅ Saved: Training/charts/{ticker}_enhanced_analysis.png")
    plt.show()
    
    # Interactive Plotly Chart
    print("📊 Creating interactive chart...")
    plot_data = data_clean.reset_index()
    
    fig2 = go.Figure()
    
    # Candlestick
    fig2.add_trace(go.Candlestick(
        x=plot_data['Date'],
        open=plot_data['Open'],
        high=plot_data['High'],
        low=plot_data['Low'],
        close=plot_data['Close'],
        name='Price',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))
    
    # Moving averages
    fig2.add_trace(go.Scatter(x=plot_data['Date'], y=plot_data['MA_50'],
                             name='MA 50', line=dict(color='red', width=2)))
    fig2.add_trace(go.Scatter(x=plot_data['Date'], y=plot_data['MA_200'],
                             name='MA 200', line=dict(color='green', width=2)))
    
    # Bollinger Bands
    fig2.add_trace(go.Scatter(x=plot_data['Date'], y=plot_data['BB_Upper_20'],
                             name='BB Upper', line=dict(color='gray', width=1, dash='dash'),
                             showlegend=True))
    fig2.add_trace(go.Scatter(x=plot_data['Date'], y=plot_data['BB_Lower_20'],
                             name='BB Lower', line=dict(color='gray', width=1, dash='dash'),
                             fill='tonexty', fillcolor='rgba(128,128,128,0.1)',
                             showlegend=True))
    
    # Add prediction points with confidence intervals
    for horizon, pred in predictions.items():
        future_date = last_date + timedelta(days=offsets[horizon])
        
        # Prediction marker
        fig2.add_trace(go.Scatter(
            x=[future_date],
            y=[pred['predicted_price']],
            mode='markers+text',
            name=f'{horizon} Prediction',
            marker=dict(size=20, symbol='star', color=colors[horizon], 
                       line=dict(color='black', width=2)),
            text=[f"${pred['predicted_price']:.2f}<br>{pred['price_change_pct']:+.2f}%"],
            textposition="top center",
            textfont=dict(size=12, color=colors[horizon], family='Arial Black')
        ))
        
        # Confidence interval
        fig2.add_trace(go.Scatter(
            x=[future_date, future_date, future_date],
            y=[pred['confidence_lower'], pred['predicted_price'], pred['confidence_upper']],
            mode='lines',
            name=f'{horizon} CI',
            line=dict(color=colors[horizon], width=0),
            showlegend=False,
            error_y=dict(
                type='data',
                symmetric=False,
                array=[pred['confidence_upper'] - pred['predicted_price']],
                arrayminus=[pred['predicted_price'] - pred['confidence_lower']],
                color=colors[horizon],
                thickness=3,
                width=10
            )
        ))
        
        # Prediction line
        fig2.add_trace(go.Scatter(
            x=[last_date, future_date],
            y=[current_price, pred['predicted_price']],
            mode='lines',
            name=f'{horizon} Path',
            line=dict(color=colors[horizon], width=3, dash='dash'),
            showlegend=False
        ))
    
    fig2.update_layout(
        title=dict(
            text=f'{ticker} - {company_name} - Interactive Technical Analysis with AI Predictions',
            font=dict(size=20, color='darkblue', family='Arial Black')
        ),
        xaxis_title='Date',
        yaxis_title='Price ($)',
        xaxis_rangeslider_visible=False,
        height=800,
        template='plotly_white',
        hovermode='x unified',
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1
        )
    )
    
    fig2.write_html(f'Training/charts/{ticker}_interactive_predictions.html')
    print(f"✅ Saved: Training/charts/{ticker}_interactive_predictions.html")
    fig2.show()
    
    print("✅ All visualizations generated successfully!")

#-------------------[MAIN EXECUTION]-----------------------#
def main():
    """Main execution function"""
    create_directories()
    
    print("\n" + "="*80)
    print("🚀 ENHANCED STOCK PRICE PREDICTION SYSTEM WITH ENSEMBLE AI")
    print("="*80)
    print("📌 Features: Multi-Model Ensemble | Advanced Technical Analysis | Confidence Intervals")
    print("="*80)
    
    # Display available stocks
    print("\n📊 AVAILABLE STOCKS FOR ANALYSIS:")
    print("-"*80)
    stocks_list = list(load_stock_list().items())
    for i in range(0, len(stocks_list), 3):
        row = stocks_list[i:i+3]
        line = "   "
        for symbol, name in row:
            line += f"{symbol:8} - {name[:30]:30}   "
        print(line)
    
    # Get user input
    print("\n" + "="*80)
    search = input("🔍 Enter stock symbol or company name (press Enter for AAPL): ").strip()
    
    if not search:
        print("✅ Using default: AAPL - Apple Inc.")
        ticker, company_name = 'AAPL', 'Apple Inc.'
    else:
        matches = find_stock(search)
        if not matches:
            print(f"❌ No matches found for '{search}'. Using AAPL as default.")
            ticker, company_name = 'AAPL', 'Apple Inc.'
        elif len(matches) == 1:
            ticker, company_name = list(matches.items())[0]
            print(f"✅ Found: {ticker} - {company_name}")
        else:
            print(f"\n🔍 Found {len(matches)} matches:")
            match_list = list(matches.items())
            for i, (symbol, name) in enumerate(match_list, 1):
                print(f"   {i}. {symbol:8} - {name}")
            
            try:
                choice = int(input(f"\n👉 Select stock (1-{len(match_list)}): "))
                if 1 <= choice <= len(match_list):
                    ticker, company_name = match_list[choice - 1]
                else:
                    print("❌ Invalid choice. Using first match.")
                    ticker, company_name = match_list[0]
            except (ValueError, IndexError):
                print("❌ Invalid input. Using first match.")
                ticker, company_name = match_list[0]
    
    print(f"\n{'='*80}")
    print(f"✅ SELECTED: {ticker} - {company_name}")
    print(f"{'='*80}")
    
    # Download data
    print("\n📥 Downloading historical stock data...")
    try:
        data = yf.download(ticker, period='5y', auto_adjust=True, progress=False)
        
        if data.empty:
            print("❌ No data downloaded. Please check your internet connection or ticker symbol.")
            return
        
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)
        
        print(f"✅ Successfully downloaded {len(data)} trading days")
        print(f"📅 Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
        
        # Save raw data
        data.to_csv(f"Training/stock_data/{ticker}_raw_data.csv")
        print(f"💾 Raw data saved: Training/stock_data/{ticker}_raw_data.csv")
        
    except Exception as e:
        print(f"❌ Error downloading data: {e}")
        return
    
    # Initialize enhanced predictor
    print("\n" + "="*80)
    print("🤖 TRAINING ENHANCED ENSEMBLE AI MODEL")
    print("="*80)
    
    predictor = EnhancedStockPricePredictor()
    predictor.ticker = ticker
    predictor.company_name = company_name
    
    # Train for different time horizons
    time_horizons = ['1_Day', '5_Days', '21_Days']
    horizon_names = ['1 Day', '1 Week', '1 Month']
    
    predictions = {}
    
    for horizon, name in zip(time_horizons, horizon_names):
        print(f"\n{'─'*80}")
        if predictor.train_ensemble_model(data, horizon):
            prediction = predictor.predict_future(data, horizon)
            if prediction:
                predictions[name] = prediction
                print(f"✅ {name} prediction completed successfully")
    
    # Display results
    print("\n" + "="*80)
    print("📈 PREDICTION RESULTS SUMMARY")
    print("="*80)
    
    if isinstance(data['Close'], pd.DataFrame):
        current_price = data['Close'].iloc[-1].values[0]
    else:
        current_price = data['Close'].iloc[-1]
    
    print(f"\n💰 Current Price: ${current_price:.2f}")
    print(f"📅 As of: {data.index[-1].strftime('%Y-%m-%d')}\n")
    
    for horizon, pred in predictions.items():
        print(f"{'─'*80}")
        print(f"🎯 {horizon.upper()} FORECAST:")
        print(f"{'─'*80}")
        print(f"   Predicted Price:    ${pred['predicted_price']:.2f}")
        print(f"   Expected Change:    ${pred['price_change']:+.2f} ({pred['price_change_pct']:+.2f}%)")
        print(f"   95% Confidence:     ${pred['confidence_lower']:.2f} - ${pred['confidence_upper']:.2f}")
        print(f"   Std Deviation:      ±${pred['prediction_std']:.2f}")
        
        # Enhanced recommendation
        pct = pred['price_change_pct']
        if pct > 10:
            rec = "🚀🚀🚀 STRONG BUY - High conviction opportunity"
        elif pct > 5:
            rec = "🚀🚀 STRONG BUY - Excellent entry point"
        elif pct > 2:
            rec = "📈 BUY - Good buying opportunity"
        elif pct > -2:
            rec = "⚖️  HOLD - Maintain current position"
        elif pct > -5:
            rec = "📉 SELL - Consider reducing position"
        elif pct > -10:
            rec = "🔻🔻 STRONG SELL - Exit recommended"
        else:
            rec = "🔻🔻🔻 STRONG SELL - Immediate exit advised"
        
        print(f"   Recommendation:     {rec}\n")
    
    # Visualizations
    print("="*80)
    print("📊 GENERATING VISUALIZATIONS")
    print("="*80)
    visualize_with_predictions(data, ticker, company_name, predictions, predictor)
    
    # Save models
    print("\n" + "="*80)
    print("💾 SAVING MODELS AND METADATA")
    print("="*80)
    
    try:
        # Save predictor
        joblib.dump(predictor, f'Training/models/{ticker}_predictor.pkl')
        print(f"✅ Model saved: Training/models/{ticker}_predictor.pkl")
        
        joblib.dump(predictor, 'Training/models/main.pkl')
        print(f"✅ Main model saved: Training/models/main.pkl")
        
        # Save metadata
        metadata = {
            'ticker': ticker,
            'company_name': company_name,
            'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'data_points': len(data),
            'date_range': f"{data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}",
            'predictions': predictions,
            'metrics': predictor.training_metrics,
            'feature_importance': predictor.feature_importance.to_dict() if predictor.feature_importance is not None else None
        }
        
        joblib.dump(metadata, f'Training/models/{ticker}_metadata.pkl')
        print(f"✅ Metadata saved: Training/models/{ticker}_metadata.pkl")
        
    except Exception as e:
        print(f"❌ Error saving models: {e}")
    
    # Final summary
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE - SUMMARY")
    print("="*80)
    print(f"📊 Stock Analyzed:        {ticker} - {company_name}")
    print(f"📈 Historical Data:       {len(data)} trading days")
    print(f"🎯 Prediction Horizons:   {len(predictions)} timeframes")
    print(f"🤖 Model Type:            Ensemble (Random Forest + Gradient Boosting + Ridge)")
    print(f"📉 Technical Indicators:  50+ advanced features")
    print(f"📊 Charts Generated:      Static PNG + Interactive HTML")
    print(f"💾 Files Saved:           {ticker}_enhanced_analysis.png, {ticker}_interactive_predictions.html")
    
    print(f"\n{'='*80}")
    print("📂 GENERATED FILES:")
    print(f"{'='*80}")
    print(f"   📊 Charts:")
    print(f"      ├─ Training/charts/{ticker}_enhanced_analysis.png")
    print(f"      └─ Training/charts/{ticker}_interactive_predictions.html")
    print(f"   💾 Data:")
    print(f"      └─ Training/stock_data/{ticker}_raw_data.csv")
    print(f"   🤖 Models:")
    print(f"      ├─ Training/models/{ticker}_predictor.pkl")
    print(f"      ├─ Training/models/{ticker}_metadata.pkl")
    print(f"      └─ Training/models/main.pkl")
    
    print(f"\n{'='*80}")
    print("🔄 QUICK RELOAD INSTRUCTIONS:")
    print(f"{'='*80}")
    print("   To reload and use this trained model:")
    print("   ```python")
    print("   predictor, predictions = load_and_predict('Training/models/main.pkl')")
    print("   # or")
    print(f"   predictor, predictions = load_and_predict('Training/models/{ticker}_predictor.pkl', '{ticker}')")
    print("   ```")
    
    print(f"\n{'='*80}")
    print("⚠️  IMPORTANT DISCLAIMER")
    print(f"{'='*80}")
    print("   This tool provides predictions based on historical patterns and technical")
    print("   analysis. Stock markets are inherently unpredictable. Past performance does")
    print("   not guarantee future results. Always conduct thorough research and consult")
    print("   with financial advisors before making investment decisions.")
    print(f"{'='*80}\n")

#-------------------[LOAD AND USE MODEL FUNCTION]-----------------------#
def load_and_predict(model_path='Training/models/main.pkl', ticker=None):
    """
    Load a saved model and make new predictions
    
    Parameters:
        model_path (str): Path to the saved model file
        ticker (str): Stock ticker symbol (optional, will use stored ticker if None)
    
    Returns:
        tuple: (predictor, predictions) or (None, None) if error
    
    Example:
        >>> predictor, predictions = load_and_predict()
        >>> predictor, predictions = load_and_predict('Training/models/AAPL_predictor.pkl', 'AAPL')
    """
    print(f"🔄 Loading model from {model_path}...")
    
    try:
        predictor = joblib.load(model_path)
        print(f"✅ Model loaded successfully!")
        
        if ticker is None:
            ticker = predictor.ticker
        
        print(f"📥 Downloading latest data for {ticker}...")
        data = yf.download(ticker, period='1y', auto_adjust=True, progress=False)
        
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)
        
        print(f"✅ Downloaded {len(data)} days of data")
        
        # Make predictions
        print(f"\n🎯 Generating predictions...")
        
        time_horizons = ['1_Day', '5_Days', '21_Days']
        horizon_names = ['1 Day', '1 Week', '1 Month']
        
        predictions = {}
        for horizon, name in zip(time_horizons, horizon_names):
            prediction = predictor.predict_future(data, horizon)
            if prediction:
                predictions[name] = prediction
        
        # Display results
        if isinstance(data['Close'], pd.DataFrame):
            current_price = data['Close'].iloc[-1].values[0]
        else:
            current_price = data['Close'].iloc[-1]
        
        print(f"\n{'='*80}")
        print(f"💰 Current Price: ${current_price:.2f}")
        print(f"📅 As of: {data.index[-1].strftime('%Y-%m-%d')}")
        print(f"{'='*80}\n")
        
        for horizon, pred in predictions.items():
            print(f"🎯 {horizon.upper()} PREDICTION:")
            print(f"   Target: ${pred['predicted_price']:.2f} | "
                  f"Change: ${pred['price_change']:+.2f} ({pred['price_change_pct']:+.2f}%)")
            print(f"   95% CI: ${pred['confidence_lower']:.2f} - ${pred['confidence_upper']:.2f}")
            
            pct = pred['price_change_pct']
            if pct > 5:
                rec = "🚀 STRONG BUY"
            elif pct > 2:
                rec = "📈 BUY"
            elif pct > -2:
                rec = "⚖️  HOLD"
            elif pct > -5:
                rec = "📉 SELL"
            else:
                rec = "🔻 STRONG SELL"
            
            print(f"   Recommendation: {rec}\n")
        
        return predictor, predictions
        
    except FileNotFoundError:
        print(f"❌ Model file not found: {model_path}")
        print(f"   Please train a model first by running main()")
        return None, None
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        import traceback
        traceback.print_exc()
        return None, None

#-------------------[RUN THE PROGRAM]-----------------------#
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⚠️  Program interrupted by user")
    except ImportError as e:
        print(f"❌ Missing required package: {e}")
        print("\n💡 Install required packages:")
        print("   pip install yfinance scikit-learn pandas numpy matplotlib plotly joblib")
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        import traceback
        traceback.print_exc()
        print("\n💡 Please check your internet connection and try again.")

✓ All directories created successfully

🚀 ENHANCED STOCK PRICE PREDICTION SYSTEM WITH ENSEMBLE AI
📌 Features: Multi-Model Ensemble | Advanced Technical Analysis | Confidence Intervals

📊 AVAILABLE STOCKS FOR ANALYSIS:
--------------------------------------------------------------------------------
   AAPL     - Apple Inc.                       MSFT     - Microsoft Corporation            GOOGL    - Alphabet Inc.                    
   AMZN     - Amazon.com Inc.                  TSLA     - Tesla Inc.                       META     - Meta Platforms Inc.              
   NVDA     - NVIDIA Corporation               JPM      - JPMorgan Chase & Co.             V        - Visa Inc.                        
   WMT      - Walmart Inc.                     DIS      - The Walt Disney Company          NFLX     - Netflix Inc.                     
   INTC     - Intel Corporation                AMD      - Advanced Micro Devices           PYPL     - PayPal Holdings Inc.             

✅ Found: AAPL - Appl

Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12280\3314353156.py", line 1128, in <module>
    main()
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12280\3314353156.py", line 909, in main
    if predictor.train_ensemble_model(data, horizon):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12280\3314353156.py", line 390, in train_ensemble_model
    directional_accuracy = (actual_direction == pred_direction).sum() / len(actual_direction) * 100
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\ops\common.py", line 76, in new_method
    return method(self, other)
           ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py", line 40, in __eq__
    return self._cmp_method(other, operator.eq)
           ^^